In [1]:
import numpy as np
import matplotlib.pyplot as plt
import NNMFit

This class is just for plotting

In [2]:
import logging
from NNMFit import logging

In [3]:
class Plot():
    def __init__(self):
        self.dark_colors=("#1f78b4","#33a02c","#e31a1c","#ff7f00","#6a3d9a","#b15928","#18B4B4","#5603AD")
        self.light_colors=("#a6cee3","#b2df8a","#fb9a99","#fdbf6f","#cab2d6","#ffff99","#93F0F0","#D5AFFE")
        self.colors=self.dark_colors+self.light_colors
        self.label_dict_NNMFit_params={'astro_norm': r'$\Phi_{isotropic}$', 
                                    'gamma_astro': r'$\gamma_{isotropic}$', 
                                    'ice_holep0': r'Ice Hole $p_0$', 
                                    'ice_holep1': r'Ice Hole $p_1$', 
                                    'ice_scat': r'Ice Scattering', 
                                    'ice_abs': r'Ice Absorption',
                                    'CR_grad': r'CR Model Interp.', 
                                    'barr_h': r'Barr H', 
                                    'barr_w': r'Barr W', 
                                    'barr_y': r'Barr Y', 
                                    'barr_z': r'Barr Z', 
                                    'conv_norm': r'$\Phi_{conventional}$',
                                    'delta_gamma': r'CR $\Delta_{\gamma}$',  
                                    'galactic_norm': r'$\Phi_{galactic}$', 
                                    'gamma_galactic': r'$\gamma_{galactic}$', 
                                    'muon_norm': r'Muon Template',
                                    'muongun_norm': r'MuonGun Norm',
                                    'prompt_norm': r'$\Phi_{prompt}$', 
                                    'dom_eff': r'DOM Efficiency',
                                    'gamma_astro_first': r'$\gamma_{isotropic,1}$'
                                }
        self.grid_alpha=0.3

    def savefig(self,path):
        plt.savefig(path,format="pdf",bbox_inches="tight")

    def set_fontsize(self,fontsize=10):
        plt.close()
        plt.rc('font', size=fontsize)
        self._plot()
    
    def set_grid_alpha(self,alpha):
        self.grid_alpha=alpha
        plt.close()
        self._plot()

def get_iterable(x):
    if type(x)==str:
        return (x,)
    else:
        return x

def get_centers_from_edges(edges):
    return (edges[:-1]+edges[1:])/2


class spectrum(Plot):
    def __init__(self,bin_edges,variable,components,hists,labels=None,baseline=None,comps_no_ratio=list(),show_errors=True,errors_alpha=1):
        super().__init__()
        
        self.variable=variable
        if self.variable=="zenith":
            self.bin_edges=np.cos(bin_edges)
        else:
            self.bin_edges=bin_edges
        self.bin_centers=get_centers_from_edges(self.bin_edges)

        self.components=components
        self.hists=hists
        self.baseline=baseline
        self.comps_no_ratio=comps_no_ratio
        if type(show_errors) == dict:
            self.show_errors=show_errors
        else:
            self.show_errors={}
            for comp in self.components:
                self.show_errors[comp]=show_errors
        self.errors_alpha=errors_alpha

        self.color_dict={   "total":"#1f78b4",
                            "total MC":"#1f78b4",
                            "data":"black",
                            "conv":"#6a3d9a",
                            "astro":"#ff7f00",
                            "prompt":"#33a02c",
                            "muon":"#e31a1c",
                            "galactic":"#18B4B4"}
        self.labels=labels
        self._plot()

    def _plot(self):
        plt.figure(dpi=200,facecolor="white")

        if self.variable=="energy":
            xlabel="Reco Energy in GeV"
        elif self.variable=="zenith":
            xlabel="cos(zenith)"
        elif self.variable=="ra":
            xlabel="Right Ascension"

        if self.baseline is None:
            hist_plot=plt.subplot(111)
            hist_plot.set_xlabel(xlabel)
        else:
            hist_plot=plt.subplot(211)
            ratio_plot=plt.subplot(212,sharex=hist_plot)
            hist_plot.tick_params('x', labelbottom=False)
            ratio_plot.set_yscale("log")
            ratio_plot.set_ylabel("Ratio")
            ratio_plot.set_ylim(0.5,2)
            ratio_plot.set_xlabel(xlabel)
            ratio_plot.set_yticks((0.6,0.7,0.8,0.9,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9),labels=(),minor=True)
            ratio_plot.set_yticks((0.5,1,2),labels=(0.5,1,2))
            ratio_plot.grid(alpha=self.grid_alpha)


        for i,comp in enumerate(self.components):
            label=self.labels[comp] if self.labels is not None else comp
            zorder=2.001 if "total" in comp else 2
            if comp in self.color_dict.keys():
                color=self.color_dict[comp]
            else:
                color=self.colors[i]
            if comp=="data":
                hist_plot.errorbar(self.bin_centers,self.hists[comp],yerr=np.sqrt(self.hists[comp]),fmt="k.",capsize=3,markersize=6,label="Data",zorder=2.002,alpha=self.errors_alpha)

            else:                
                hist_plot.stairs(self.hists[comp],self.bin_edges,label=label,baseline=None,color=color,zorder=zorder)
                if self.show_errors[comp]:
                    hist_plot.errorbar(self.bin_centers,self.hists[comp],yerr=np.sqrt(self.hists[comp]),fmt="none",capsize=3,ecolor=color,linewidth=1,zorder=zorder,alpha=self.errors_alpha)


            if (self.baseline is not None) and (comp not in self.comps_no_ratio):
                baseline_hist=self.hists[self.baseline]
                if comp=="data":
                    ratio_plot.errorbar(self.bin_centers,self.hists[comp]/baseline_hist,yerr=np.sqrt(self.hists[comp])/baseline_hist,fmt="k.",capsize=3,markersize=6,alpha=self.errors_alpha)

                else:                
                    ratio_plot.stairs(self.hists[comp]/baseline_hist,self.bin_edges,baseline=None,color=color)
                    if self.show_errors[comp]:
                        ratio_plot.errorbar(self.bin_centers,self.hists[comp]/baseline_hist,yerr=np.sqrt(self.hists[comp])/baseline_hist,fmt="none",capsize=3,ecolor=color,linewidth=1,alpha=self.errors_alpha)
        
        if self.variable=="energy":
            hist_plot.set_xscale("log")
        hist_plot.set_yscale("log")
        hist_plot.set_ylabel(r"$N_{events}$")
        hist_plot.grid(alpha=self.grid_alpha)
        hist_plot.legend()

Here you set up the graph. You probably want to exchange some config files. To only exchange the MC you can use something like "/home/jhellrung/NNMFit_stuff/configs/override_configs/SnowStorm_Baseline.cfg"

In [4]:
import pandas as pd

In [5]:
config_dict=NNMFit.AnalysisConfig.from_configs(
        "/home/ssclafani/GP_Diffuse/configs/main_configs/ftp_snowstorm.cfg",
        "/home/ssclafani/GP_Diffuse/configs/analysis_configs/asimov_Globalfit_Poisson.yaml",
        override_config_files=["/home/ssclafani/GP_Diffuse/configs/override_configs/2D_to_3DBinning.cfg"],
        override_dict=None,
        config_dir="/home/ssclafani/GP_Diffuse/configs",
        override_components_files=None,
        override_parameters_files=None,
    ).to_dict()
print(config_dict)
print(config_dict['config']['IC86_pass2_SnowStorm_DNNCascade_var_mapping'])

hist_graph=NNMFit.utilities.HistogramGraph.from_configdict(config_dict)

{'config': {'main': {'dir_datasets': '/data/user/ssclafani/GP_Diffuse/NNMFit/datasets/baseline', 'dir_systematics': '/data/user/ssclafani/GP_Diffuse/NNMFit/datasets/baseline', 'caching_dir': '/data/user/ssclafani/GP_Diffuse/NNMFit/cache', 'components': 'conventional, astro, galactictemplate_Cringe', 'systematics_config': 'SnowStorm_systematics'}, 'minimizer_settings': {'class': 'LBFGSB', 'tolerance': '10'}, 'IC86_pass2_SnowStorm_DNNCascade': {'analysis_binning': 'IC86_pass2_SnowStorm_DNNCascade_2D_to_3D_binning', 'name': 'IC86_pass2_SnowStorm_DNNCascade', 'datasets_path': '/data/user/ssclafani/GP_Diffuse/NNMFit/datasets/baseline', 'baseline_dataset': '/data/user/ssclafani/GP_Diffuse/NNMFit/datasets/baseline', 'systematics': 'IC86_pass2_SnowStorm_DNNCascade_systematics', 'var_mapping': 'IC86_pass2_SnowStorm_DNNCascade_var_mapping', 'var_mapping_mc': 'IC86_pass2_SnowStorm_DNNCascade_var_mapping_mc', 'data': '', 'livetime': '387231573.49', 'modification_hooks': '', 'excluded_components': 

/data/user/ssclafani/software/venvs/nnm_venv/lib/python3.10/site-packages/NNMFit/core/graph_builder.py:137: DeprecationWarning: invalid escape sequence '\m'
  """
/data/user/ssclafani/software/venvs/nnm_venv/lib/python3.10/site-packages/NNMFit/core/histogram_builder.py:17: DeprecationWarning: invalid escape sequence '\m'
  '''
/data/user/ssclafani/software/venvs/nnm_venv/lib/python3.10/site-packages/NNMFit/core/histogram_builder.py:63: DeprecationWarning: invalid escape sequence '\m'
  """
/data/user/ssclafani/software/venvs/nnm_venv/lib/python3.10/site-packages/NNMFit/core/histogram_builder.py:99: DeprecationWarning: invalid escape sequence '\m'
  """
/data/user/ssclafani/software/venvs/nnm_venv/lib/python3.10/site-packages/NNMFit/core/histogram_builder.py:147: DeprecationWarning: invalid escape sequence '\m'
  """
/data/user/ssclafani/software/venvs/nnm_venv/lib/python3.10/site-packages/NNMFit/core/histogram_builder.py:223: DeprecationWarning: invalid escape sequence '\s'
  """
/data

ModuleNotFoundError: No module named 'icecube._dataclasses'

Initialize different parameter combinations

In [ ]:
input_total={"conv_norm":1.0,
 "delta_gamma":0.0,
 "CR_grad":0.0,
 "barr_h":0.0,
 "barr_w":0.0,
 "barr_z":0.0,
 "barr_y":0.0,
 "astro_norm":1.8,
 "gamma_astro":2.52,
 "prompt_norm":0.0,
 "muon_norm":1.0,
 "muongun_norm":1.0,
 "galactic_norm":4.7,
 "dom_eff":0.0}

input_conv={"conv_norm":1.0,
 "delta_gamma":0.0,
 "CR_grad":0.0,
 "barr_h":0.0,
 "barr_w":0.0,
 "barr_z":0.0,
 "barr_y":0.0,
 "astro_norm":0.0,
 "gamma_astro":2.52,
 "prompt_norm":0.0,
 "muon_norm":0.0,
 "muongun_norm":0.0,
 "galactic_norm":0.0}

input_astro={"conv_norm":0.0,
 "delta_gamma":0.0,
 "CR_grad":0.0,
 "barr_h":0.0,
 "barr_w":0.0,
 "barr_z":0.0,
 "barr_y":0.0,
 "astro_norm":1.8,
 "gamma_astro":2.52,
 "prompt_norm":0.0,
 "muon_norm":0.0,
 "muongun_norm":0.0,
 "galactic_norm":0.0}

input_prompt={"conv_norm":0.0,
 "delta_gamma":0.0,
 "CR_grad":0.0,
 "barr_h":0.0,
 "barr_w":0.0,
 "barr_z":0.0,
 "barr_y":0.0,
 "astro_norm":0.0,
 "gamma_astro":2.52,
 "prompt_norm":1.0,
 "muon_norm":0.0,
 "muongun_norm":0.0,
 "galactic_norm":0.0}

input_muon={"conv_norm":0.0,
 "delta_gamma":0.0,
 "CR_grad":0.0,
 "barr_h":0.0,
 "barr_w":0.0,
 "barr_z":0.0,
 "barr_y":0.0,
 "astro_norm":0.0,
 "gamma_astro":2.52,
 "prompt_norm":0.0,
 "muon_norm":1.0,
 "muongun_norm":1.0,
 "galactic_norm":0.0}

input_galactic={"conv_norm":0.0,
 "delta_gamma":0.0,
 "CR_grad":0.0,
 "barr_h":0.0,
 "barr_w":0.0,
 "barr_z":0.0,
 "barr_y":0.0,
 "astro_norm":0.0,
 "gamma_astro":2.52,
 "prompt_norm":0.0,
 "muon_norm":0.0,
 "muongun_norm":0.0,
 "galactic_norm":4.7}

In [ ]:
sys_dict={
    "dom_eff":1.0,
    "ice_abs":1.0,
    "ice_scat":1.0,
    "ice_holep0":-0.27,
    "ice_holep1": -0.042
}


Calculate Histograms

In [ ]:
input_dict={"total":input_total,"conv":input_conv,"astro":input_astro,"prompt":input_prompt,"galactic":input_galactic,"muon":input_muon}
hist_dict={}
for det_config in ("IC86_pass2_SnowStorm_tracks","IC86_pass2_SnowStorm_DNNCascade"):
    hist_dict[det_config]={}
    for component in input_dict.keys():
        input_variables=input_dict[component]|sys_dict
        hist=hist_graph.get_evaled_histogram(input_variables,det_config=det_config)
        hist_dict[det_config][component]=hist["mu"].reshape(50,33,180) if "tracks" in det_config else hist["mu"].reshape(22,16,72)
    hist_dict[det_config]["bin_edges"]=hist["bin_edges"]

Plot Histograms

In [ ]:
for det_config in ("IC86_pass2_SnowStorm_tracks","IC86_pass2_SnowStorm_DNNCascade"):
    for i,var in enumerate(("energy","zenith","ra")):
        hists_to_plot={}
        axis=[0,1,2]
        axis.remove(i)
        axis=tuple(axis)
        for comp in input_dict.keys():
            hists_to_plot[comp]=hist_dict[det_config][comp].sum(axis=axis)
        spec=spectrum(hist_dict[det_config]["bin_edges"]["reco_"+var],var,input_dict.keys(),hists_to_plot,baseline=None,comps_no_ratio=list(),show_errors=True,errors_alpha=0.3)
        spec.set_grid_alpha(0.7)

This is only for the systematics

In [ ]:

hist_dict_sys={}
mods={
    "dom_eff":{"low":0.95,"default":1.0,"high":1.05},
    "ice_abs":{"low":0.95,"default":1.0,"high":1.05},
    "ice_scat":{"low":0.95,"default":1.0,"high":1.05},
    "ice_holep0":{"low":-0.47,"default":-0.27,"high":0.13},
    "ice_holep1":{"low":-0.142,"default":-0.042,"high":0.048}
}

for det_config in ("IC86_pass2_SnowStorm_tracks","IC86_pass2_SnowStorm_DNNCascade"):
    hist_dict_sys[det_config]={}
    for sys_par in sys_dict.keys():
        hist_dict_sys[det_config][sys_par]={}
        for mod in ("low","default","high"):
            sys_dict_mod=sys_dict
            sys_dict_mod[sys_par]=mods[sys_par][mod]
            input_variables=input_total|sys_dict_mod
            hist=hist_graph.get_evaled_histogram(input_variables,det_config=det_config)
            hist_dict_sys[det_config][sys_par][mod]=hist["mu"].reshape(50,33,180) if "tracks" in det_config else hist["mu"].reshape(22,16,72)
    hist_dict_sys[det_config]["bin_edges"]=hist["bin_edges"]

    for i,var in enumerate(("energy","zenith","ra")):
        for sys_par in sys_dict.keys():
            hists_to_plot={}
            axis=[0,1,2]
            axis.remove(i)
            axis=tuple(axis)
            for mod in ("low","default","high"):
                hists_to_plot[mod]=hist_dict_sys[det_config][sys_par][mod].sum(axis=axis)
            spectrum(hist_dict_sys[det_config]["bin_edges"]["reco_"+var],var,("low","default","high"),hists_to_plot,baseline="default",comps_no_ratio=list(),show_errors=True)
            plt.title(sys_par)
        